In [1]:
import pandas as pd
import numpy as np
import warnings

# Tắt các cảnh báo không cần thiết
warnings.filterwarnings('ignore')

In [ ]:
def load_data(filepath):
    """Load dữ liệu từ file CSV"""
    print(f"Đang tải dữ liệu từ: {filepath}...")
    try:
        df = pd.read_csv(filepath)
        print(f"Đã tải thành công! Kích thước gốc: {df.shape}")
        return df
    except Exception as e:
        print(f"Lỗi tải file: {e}")
        return None


In [5]:
def clean_data(df):
    # 1. Chuẩn hóa tên cột (đổi dấu chấm thành gạch dưới về chữ thường)
    df.columns = df.columns.str.replace('.', '_').str.lower()
    print("- Đã chuẩn hóa tên cột (ví dụ: 'Order.ID' -> 'order_id').")

    # 2. Xóa cột rác (nếu có)
    junk_cols = ['row_id', '记录数', 'weeknum', 'market2']
    cols_to_drop = [c for c in junk_cols if c in df.columns]
    if cols_to_drop:
        df.drop(columns=cols_to_drop, inplace=True)
        print(f"- Đã xóa các cột thừa: {cols_to_drop}")

    # 3. Chuyển đổi định dạng ngày tháng
    df['order_date'] = pd.to_datetime(df['order_date'])
    df['ship_date'] = pd.to_datetime(df['ship_date'])
    print("- Đã chuyển 'order_date' và 'ship_date' sang dạng Datetime.")

    # 4. Xử lý Logic Nghiệp vụ (Phần bạn muốn chuyển sang đây)
    # 4.1. Xóa đơn hàng có Ngày Ship < Ngày Đặt (Vô lý)
    invalid_dates = df[df['ship_date'] < df['order_date']]
    if not invalid_dates.empty:
        print(f"- ⚠️ Phát hiện {len(invalid_dates)} dòng lỗi (Ship < Order). Đang xóa...")
        df = df[df['ship_date'] >= df['order_date']]
    
    # 4.2. Xử lý trùng lặp (Business Key: Order ID + Product ID)
    # Group lại và cộng dồn các chỉ số số lượng
    # Lưu ý: Ở đây ta giả định đơn giản là xóa trùng lặp hoàn toàn trước
    before_dedup = len(df)
    df = df.drop_duplicates()
    after_dedup = len(df)
    if before_dedup > after_dedup:
        print(f"- Đã xóa {before_dedup - after_dedup} dòng trùng lặp hoàn toàn.")

    return df

In [ ]:
def main():
    # PATH CONFIG (Cấu hình đường dẫn)
    INPUT_PATH = '../data/raw/superstore.csv' # File gốc
    OUTPUT_PATH = '../data/processed/superstore_final.csv' # File thành phẩm
    
    # 1. LOAD
    try:
        raw_df = load_data(INPUT_PATH)
    except:
        raw_df = load_data('superstore.csv')

    if raw_df is not None:
    # 2. CLEAN
        clean_df = clean_data(raw_df)
        print(f"\n💾 Đang lưu file processed xuống: {OUTPUT_PATH}")
        print("✅ Hoàn tất! Dữ liệu đã sẵn sàng cho Tableau.")

if __name__ == "__main__":
    main()

⏳ Đang tải dữ liệu từ: ../data/raw/superstore.csv...
Đã tải thành công! Kích thước gốc: (51290, 27)
- Đã chuẩn hóa tên cột (ví dụ: 'Order.ID' -> 'order_id').
- Đã xóa các cột thừa: ['row_id', '记录数', 'weeknum', 'market2']
- Đã chuyển 'order_date' và 'ship_date' sang dạng Datetime.

--- BẮT ĐẦU FEATURE ENGINEERING ---
- Đã tạo cột: 'delivery_days', 'is_late'
- Đã tạo cột: 'profit_margin', 'is_loss_maker'
- Đã tạo cột: 'acquisition_date', 'cohort_month' (cho Cohort Analysis)
- Đã tạo cột: 'order_year', 'order_month', 'order_quarter'
- Đã tạo cột: 'discount_level'

--- KẾT QUẢ CUỐI CÙNG ---
<class 'pandas.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   category          51290 non-null  str           
 1   city              51290 non-null  str           
 2   country           51290 non-null  str           
 3   customer_id       51290 non-null 

: 